In [1]:
import numpy as np
import os

timestamps = np.load(os.path.join('dump', 'timestamps.npy'))
pydatetime = np.load(os.path.join('dump', 'pydatetime.npy'))
radiometry = np.load(os.path.join('dump', 'radiometry.npy'))
localmeteo = np.load(os.path.join('dump', 'localmeteo.npy'))
meteosonde = np.load(os.path.join('dump', 'meteosonde.npy'))

In [ ]:
import dill
import os

with open(os.path.join('dump', 'timestamps.dump'), 'rb') as timestamps_dump:
    timestamps = dill.load(timestamps_dump)
with open(os.path.join('dump', 'pydatetime.dump'), 'rb') as pydatetime_dump:
    pydatetime = dill.load(pydatetime_dump)
with open(os.path.join('dump', 'radiometry.dump'), 'rb') as radiometry_dump:
    radiometry = dill.load(radiometry_dump)
with open(os.path.join('dump', 'localmeteo.dump'), 'rb') as localmeteo_dump:
    localmeteo = dill.load(localmeteo_dump)
with open(os.path.join('dump', 'meteosonde.dump'), 'rb') as meteosonde_dump:
    meteosonde = dill.load(meteosonde_dump)

In [2]:
print(
    timestamps.shape, timestamps.dtype, '\n',
    pydatetime.shape, pydatetime.dtype, '\n',
    radiometry.shape, radiometry.dtype, '\n',
    localmeteo.shape, localmeteo.dtype, '\n',
    meteosonde.shape, meteosonde.dtype, '\n',
)

(8547172,) float64 
 (8547172, 7) int64 
 (8547172, 47) float32 
 (8547172, 5) float32 
 (8547172, 4) int64 



In [3]:
channels = {}
for i, f in enumerate(np.linspace(18, 27.2, 47, endpoint=True)):
    channels[np.round(f, decimals=1)] = i

In [ ]:
radiometry[0, :][channels[18.0]]

In [4]:
import dill
with open('Dolgoprudnyj.dump', 'rb') as dump:
    meteosonde_data = dill.load(dump)

In [5]:
import sys
sys.path.append('../atmrad')

In [6]:
from cpu.atmosphere import Atmosphere, avg
from cpu.weight_funcs import krho
from cpu.core.static.weight_funcs import kw

In [20]:
np.unique(localmeteo, axis=0).shape

(1011857, 5)

In [15]:
T_cosmic = 2.72548

# radiometry_angle = 51.*np.pi/180
radiometry_angle = 0.

resonance_channel = 22.2
second_channel = [18, 27.2]
frequency_pairs = [(resonance_channel, sc) for sc in second_channel]

_Q_real_, _Q_std_ = [], []
_Q_retr_, _W_retr_ = [], []

k_w_std = [np.asarray([kw(nu, t=0.) for nu in freq_pair]) for freq_pair in frequency_pairs]

for i in range(len(meteosonde)):
    
    BT_spectra = radiometry[i, :]
    
    T, P, rho_rel, alt = meteosonde_data[tuple(meteosonde[i])]
    Atm_real = Atmosphere(T, P, RelativeHumidity=rho_rel, altitudes=alt)
    T0, P0, rho0, *_ = localmeteo[i]
    Atm_std = Atmosphere.Standard(T0=T0, P0=P0, rho0=rho0)
    
    Q_real = Atm_real.Q  # real TWV
    Q_std = Atm_std.Q  # TWV of standard atmosphere
    
    _Q_real_.append(Q_real)
    _Q_std_.append(Q_std)
    
    Q_retr_, W_retr_ = [], []
    for j, freq_pair in enumerate(frequency_pairs):
        
        Tb_measured = np.asarray([BT_spectra[channels[nu]] for nu in freq_pair])  # под углом

        Tavg_down_std = np.asarray([avg.downward.T(Atm_std, nu, radiometry_angle) 
                                    for nu in freq_pair])  # под углом 
        
        
        k_rho_std = np.asarray([krho(Atm_std, nu) for nu in freq_pair])  # в зените
        
        M = np.transpose([k_rho_std, k_w_std[j]])
        
        
        tau_o_std = np.asarray([Atm_std.opacity.oxygen(nu) for nu in freq_pair])  # в зените
        
        
        tau_e = np.log((Tavg_down_std - T_cosmic) / (Tavg_down_std - Tb_measured)) * np.cos(radiometry_angle)  # в зените
        
        Q_retr, W_retr = np.linalg.solve(M, tau_e - tau_o_std)
        
        Q_retr_.append(Q_retr)
        W_retr_.append(W_retr)
    
    _Q_retr_.append(Q_retr_)
    _W_retr_.append(W_retr_)
     
#     if i % 10000 == 0:
    print('\r{:.5f}%'.format((i+1) / len(localmeteo) * 100), end='      ', flush=True)
        
_Q_real_, _Q_std_ = np.asarray(_Q_real_, dtype=np.float32), np.asarray(_Q_std_, dtype=np.float32)
_Q_retr_, _W_retr_ = np.asarray(_Q_retr_, dtype=np.float32), np.asarray(_W_retr_, dtype=np.float32)


0.00647%      

KeyboardInterrupt: 